In [24]:
# Общие библиотеки
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
import os
from sklearn.model_selection import train_test_split
import json
from math import sin, cos
import random

# Для создания и обучения модели
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms

# Для работы с изображениями
import cv2
from PIL import Image, ImageTransform

# Для визуализации
import matplotlib.pyplot as plt
from IPython.display import clear_output

Преобразование начального датасета

На данном этапе данные, хранящиеся в json файлах, преобразовываются в pandas-датафрейм.

In [ ]:
json_dir = "/content/drive/MyDrive/train/json/"

data_df = pd.DataFrame({'id': [], "left_top_x": [], 'left_top_y': [], "right_bottom_x": [], 'right_bottom_y': [], 'angle': []})

json_true = []
for _, _, files in os.walk(json_dir):
    for x in files:
        if x.endswith(".json"):
            data = json.load(open(json_dir + x))
            new_row = {'id':x.split(".")[0]+".png", 'left_top_x':data["left_top"][0], 'left_top_y':data["left_top"][1], 'right_bottom_x': data["right_bottom"][0], "right_bottom_y": data["right_bottom"][1], 'angle': data["angle"]}
            data_df = data_df.append(new_row, ignore_index=True)

data_df.head(5)

,id,left_top_x,left_top_y,right_bottom_x,right_bottom_y,angle
0,1.png,8533.0,2184.0,9834.0,2819.0,341.0
1,41.png,3190.0,4252.0,1889.0,3617.0,161.0
2,30.png,2690.0,7360.0,3015.0,8771.0,32.0
3,7.png,9609.0,6995.0,8484.0,7906.0,96.0
4,71.png,6535.0,1578.0,5184.0,2097.0,114.0


In [ ]:
data_df_s = pd.DataFrame({'id': [], "left_top_x": [], 'left_top_y': [], 'left_bottom_x': [],  'left_bottom_y': [], 'right_bottom_x': [], 'right_bottom_y': [], 'angle': []})

json_true = []
for _, _, files in os.walk(json_dir):
    for x in files:
        if x.endswith(".json"):
            data = json.load(open(json_dir + x))
            new_row = {'id':x.split(".")[0]+".png", 'left_top_x':data["left_top"][0], 'left_top_y':data["left_top"][1], 'left_bottom_x': data["left_bottom"][0], 'left_bottom_y': data["left_bottom"][1], 'right_bottom_x': data["right_bottom"][0], "right_bottom_y": data["right_bottom"][1], 'angle': data["angle"]}
            data_df_s = data_df_s.append(new_row, ignore_index=True)

data_df_s.head(5)

,id,left_top_x,left_top_y,left_bottom_x,left_bottom_y,right_bottom_x,right_bottom_y,angle
0,1.png,8533.0,2184.0,8866.0,3152.0,9834.0,2819.0,341.0
1,41.png,3190.0,4252.0,2857.0,3284.0,1889.0,3617.0,161.0
2,30.png,2690.0,7360.0,2147.0,8228.0,3015.0,8771.0,32.0
3,7.png,9609.0,6995.0,8591.0,6888.0,8484.0,7906.0,96.0
4,71.png,6535.0,1578.0,5600.0,1162.0,5184.0,2097.0,114.0


In [ ]:
s_df = data_df_s.copy()
s_df['a'] = (abs(data_df_s['left_top_x'] - data_df_s['left_bottom_x']) ** 2 + abs(data_df_s['left_top_y'] - data_df_s['left_bottom_y']) ** 2) ** 0.5
s_df['b'] = (abs(data_df_s['right_bottom_x'] - data_df_s['left_bottom_x']) ** 2 + abs(data_df_s['right_bottom_y'] - data_df_s['left_bottom_y']) ** 2) ** 0.5
s_df['square'] = s_df['a'] * s_df['b']
s_df

,id,left_top_x,left_top_y,left_bottom_x,left_bottom_y,right_bottom_x,right_bottom_y,angle,a,b,square
0,1.png,8533.0,2184.0,8866.0,3152.0,9834.0,2819.0,341.0,1023.676218,1023.676218,1047913.0
1,41.png,3190.0,4252.0,2857.0,3284.0,1889.0,3617.0,161.0,1023.676218,1023.676218,1047913.0
2,30.png,2690.0,7360.0,2147.0,8228.0,3015.0,8771.0,32.0,1023.852040,1023.852040,1048273.0
3,7.png,9609.0,6995.0,8591.0,6888.0,8484.0,7906.0,96.0,1023.607835,1023.607835,1047773.0
4,71.png,6535.0,1578.0,5600.0,1162.0,5184.0,2097.0,114.0,1023.367480,1023.367480,1047281.0
...,...,...,...,...,...,...,...,...,...,...,...
795,1112.png,4492.0,5122.0,3580.0,5587.0,4045.0,6499.0,63.0,1023.703570,1023.703570,1047969.0
796,1136.png,8010.0,6727.0,8878.0,6184.0,8335.0,5316.0,238.0,1023.852040,1023.852040,1048273.0
797,1113.png,5638.0,7815.0,4653.0,8097.0,4935.0,9082.0,74.0,1024.572594,1024.572594,1049749.0
798,1161.png,5653.0,3042.0,4995.0,2257.0,4210.0,2915.0,140.0,1024.299273,1024.299273,1049189.0


In [ ]:
im = Image.open('/content/drive/MyDrive/train/original.tiff')
im_crop_outside = im.crop((6934.0, 8975.0, 7958.0, 9999.0))
im_crop_outside.show()

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (110166016 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


In [ ]:


trans = [5358, 5282, 4634, 4557, 3909, 5282, 4634, 6006]
im.transform((1024, 1024), ImageTransform.QuadTransform(trans))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
s_df['center_x'] = ((s_df['left_top_x'] + s_df['right_bottom_x']) / 2).astype(np.int32)
s_df['center_y'] = ((s_df['left_top_y'] + s_df['right_bottom_y']) / 2).astype(np.int32)
s_df

,id,left_top_x,left_top_y,left_bottom_x,left_bottom_y,right_bottom_x,right_bottom_y,angle,a,b,square,center_x,center_y
0,1.png,8533.0,2184.0,8866.0,3152.0,9834.0,2819.0,341.0,1023.676218,1023.676218,1047913.0,9183,2501
1,41.png,3190.0,4252.0,2857.0,3284.0,1889.0,3617.0,161.0,1023.676218,1023.676218,1047913.0,2539,3934
2,30.png,2690.0,7360.0,2147.0,8228.0,3015.0,8771.0,32.0,1023.852040,1023.852040,1048273.0,2852,8065
3,7.png,9609.0,6995.0,8591.0,6888.0,8484.0,7906.0,96.0,1023.607835,1023.607835,1047773.0,9046,7450
4,71.png,6535.0,1578.0,5600.0,1162.0,5184.0,2097.0,114.0,1023.367480,1023.367480,1047281.0,5859,1837
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1112.png,4492.0,5122.0,3580.0,5587.0,4045.0,6499.0,63.0,1023.703570,1023.703570,1047969.0,4268,5810
796,1136.png,8010.0,6727.0,8878.0,6184.0,8335.0,5316.0,238.0,1023.852040,1023.852040,1048273.0,8172,6021
797,1113.png,5638.0,7815.0,4653.0,8097.0,4935.0,9082.0,74.0,1024.572594,1024.572594,1049749.0,5286,8448
798,1161.png,5653.0,3042.0,4995.0,2257.0,4210.0,2915.0,140.0,1024.299273,1024.299273,1049189.0,4931,2978


In [ ]:
s_df[s_df['id'] == '71.png']

,id,left_top_x,left_top_y,left_bottom_x,left_bottom_y,right_bottom_x,right_bottom_y,angle,a,b,square,center_x,center_y
4,71.png,6535.0,1578.0,5600.0,1162.0,5184.0,2097.0,114.0,1023.36748,1023.36748,1047281.0,5859,1837


In [53]:

def rotate(p, origin=(0, 0), degrees=0):
    '''Считаем координаты прямоугольника после поворота на заданный угол'''
    angle = np.deg2rad(degrees)
    R = np.array([[np.cos(angle), -np.sin(angle)],
                  [np.sin(angle),  np.cos(angle)]])
    o = np.atleast_2d(origin)
    p = np.atleast_2d(p)

    return np.squeeze((R @ (p.T-o.T) + o.T).T).astype(np.int32)

def get_new_coord(center_of_rectangle, degree = 0):
  """Считаем координаты прямоугольника с учетом центра и угла поворота
  
  Arguments:
  center_of_rectangle - (x, y) center coordinates,
  degree - angle of rotation of the rectangle

  Returns: (left_top_x, left_top_y), (right_top_x, right_top_y), (left_bottom_x, left_bottom_y), (right_bottom_x, right_bottom_y) 
  """

  left_top = (center_of_rectangle[0] - 512, center_of_rectangle[1] + 512)
  right_top = (center_of_rectangle[0] + 512, center_of_rectangle[1] + 512)
  left_bottom = (center_of_rectangle[0] - 512, center_of_rectangle[1] - 512)
  right_bottom = (center_of_rectangle[0] + 512, center_of_rectangle[1] - 512)

  points=[left_top, right_top, left_bottom, right_bottom]

  new_points = rotate(points, origin= center_of_rectangle, degrees= -degree)

  new_points[0][1], new_points[3][1] = new_points[3][1], new_points[0][1]
  new_points[1][1], new_points[2][1] = new_points[2][1], new_points[1][1]

  #left_top, right_top, left_bottom, right_bottom
  return new_points[0], new_points[1], new_points[2], new_points[3] 

1) Угол ставим отрицательный

2) Поменять местами left_top_y и right_bottom_y;
right_top_y и left_bottom_y

{"left_top": [8533, 2184], 
"right_top": [9501, 1851], 
"left_bottom": [8866, 3152], 
"right_bottom": [9834, 2819], "angle": 341}

[[**8532** 2818]
 [**9500** 3151]
 [**8865** 1850]
 [**9833** 2183]]

 {"left_top": [3190, 4252], 
 "right_top": [2222, 4585], 
 "left_bottom": [2857, 3284], 
 "right_bottom": [1889, 3617], "angle": 161}

 [[**3189** 3616]
 [**2221** 3283]
 [**2856** 4584]
 [**1888** 4251]]

In [ ]:
center_x, center_y = s_df[s_df['id'] == '71.png'].center_x.values[0], s_df[s_df['id'] == '71.png'].center_y.values[0]
angle = s_df[s_df['id'] == '71.png'].angle.values[0]
left_top, right_top, left_bottom, right_bottom = get_new_coord((center_x, center_y), angle)
trans = [left_top[0], left_top[1], left_bottom[0], left_bottom[1], right_bottom[0], right_bottom[1], right_top[0], right_top[1] ]
im.transform((1024, 1024), ImageTransform.QuadTransform(trans))

Output hidden; open in https://colab.research.google.com to view.

**Работает!!!**

In [57]:
def generate_train_sample(im, image_dir, index_str, json_dir):
  '''Функция обрезает исходное изображение и создает json файл'''
  flag = False
  while flag == False:

    # Генерируем случайные значения координат центра и угла поворота
    x_center = random.randint(512, 10496-512)
    y_center = random.randint(512, 10496-512)
    angle = random.randint(0, 359)

    # Получаем края прямоугольника
    left_top, right_top, left_bottom, right_bottom = get_new_coord((x_center, y_center), angle)
    #print('Получили края')

    # Проверяем, чтобы все края прямоугольников не выходили за границу
    for point in [left_top, right_top, left_bottom, right_bottom]:
      for coordinate in point:
        if coordinate > 10496:
          continue
    flag = True

  # Вырезаем картинку
  trans = [left_top[0], left_top[1], left_bottom[0], left_bottom[1], right_bottom[0], right_bottom[1], right_top[0], right_top[1] ]
  im_crop = im.transform((1024, 1024), ImageTransform.QuadTransform(trans))


  # Сохраняем картинку
  im_crop.save(image_dir + index_str + '.png')

  #print('Сохранили картинку')

  res = {
        'left_top': [int(left_top[0]), int(left_top[1])],
        'right_top': [int(right_top[0]), int(right_top[1])],
        'left_bottom': [int(left_bottom[0]), int(left_bottom[1])],
        'right_bottom': [int(right_bottom[0]), int(right_bottom[1])],
        'angle': angle
         }

  # Сохраняем json
  with open(json_dir + index_str + '.json', 'w') as f:
    json.dump(res, f)
  print('1', end = '')

In [55]:
json_dir = '/content/drive/MyDrive/train/test_jsons/'

angle = 30
left_top, right_top, left_bottom, right_bottom = get_new_coord((5000, 5000), angle)
res = {
        'left_top': [int(left_top[0]), int(left_top[1])],
        'right_top': [int(right_top[0]), int(right_top[1])],
        'left_bottom': [int(left_bottom[0]), int(left_bottom[1])],
        'right_bottom': [int(right_bottom[0]), int(right_bottom[1])],
        'angle': angle
         }
print('Создали json: ', left_top, right_top, left_bottom, right_bottom)
# Сохраняем json
with open(json_dir + '0' + '.json', 'w') as f:
  json.dump(res, f)

Создали json:  [4812 4300] [5699 4812] [4300 5187] [5187 5699]


In [58]:
im = Image.open('/content/drive/MyDrive/train/original.tiff')
image_dir = '/content/drive/MyDrive/train/img/'
json_dir = '/content/drive/MyDrive/train/json/'

#index = str(1) + '_gen'
#generate_train_sample(im, image_dir, index, json_dir)

for i in range(200, 800):
  index = str(i) + '_gen'
  generate_train_sample(im, image_dir, index, json_dir)

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (110166016 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [4]:
# Для работы с изображениями
import cv2
from PIL import Image, ImageTransform

In [8]:
import random


# Генерируем облака

In [1]:
!git clone https://github.com/aleju/imgaug.git

Cloning into 'imgaug'...
remote: Enumerating objects: 17405, done.
remote: Total 17405 (delta 0), reused 0 (delta 0), pack-reused 17405
Receiving objects: 100% (17405/17405), 27.86 MiB | 18.77 MiB/s, done.
Resolving deltas: 100% (13292/13292), done.


In [2]:
import imgaug.augmenters as iaa

In [14]:
# Инициализируем разные варианты аугментации. С двумя, 3 и 4 слоями облаков соответственно
seq = [iaa.Sequential([iaa.Clouds(), iaa.Clouds()]), 
       iaa.Sequential([iaa.Clouds(), iaa.Clouds(), iaa.Clouds()]), 
       iaa.Sequential([iaa.Clouds(), iaa.Clouds(), iaa.Clouds(), iaa.Clouds()])
       ]

# Изменяем сгенерированные ранее изображения на те же изображения, но с облаками

for index in range(51, 800, 10):
  # Генерируем случайное число -  вариант аугментации
  rand_int = random.randint(0, 2)
  imglist = []

  for i in range(0, 10):
    file_path = '/content/drive/MyDrive/train/img/' + str(index + i) + '_gen.png'
    img = cv2.imread(file_path)
    imglist.append(img)

  images_aug = seq[rand_int].augment_images(imglist)

  for i in range(0, 10):
    file_path = '/content/drive/MyDrive/train/img/' + str(index + i) + '_gen.png'
    cv2.imwrite(file_path, images_aug[i - 1])
  
  print(f'{index}-{index + 9}', end = ' ')

  #cv2.imwrite(file_path, images_aug[0])

51-60 61-70 71-80 81-90 91-100 101-110 111-120 121-130 131-140 141-150 151-160 161-170 171-180 181-190 191-200 201-210 211-220 221-230 231-240 241-250 251-260 261-270 271-280 281-290 291-300 301-310 311-320 321-330 331-340 341-350 351-360 361-370 371-380 381-390 391-400 401-410 411-420 421-430 431-440 441-450 451-460 461-470 471-480 481-490 491-500 501-510 511-520 521-530 531-540 541-550 551-560 561-570 571-580 581-590 591-600 601-610 611-620 621-630 631-640 641-650 651-660 661-670 671-680 681-690 691-700 701-710 711-720 721-730 731-740 741-750 751-760 761-770 771-780 781-790 

AttributeError: ignored

In [10]:
len(images_aug)

1

In [ ]:
im_crop_outside.save('/content/drive/MyDrive/train/test_img_1079.png')